In [ ]:
!pip install torchxrayvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.0/29.0 MB 7.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
folder_path_frontal = '/content/drive/My Drive/NLMCXR/frontal_images'
folder_path_lateral = '/content/drive/My Drive/NLMCXR/lateral_images'
import os
files = os.listdir(folder_path_frontal)
print(files)

['2975.png', '2949.png', '2960.png', '2955.png', '2976.png', '2946.png', '2970.png', '2965.png', '2982.png', '2974.png', '2977.png', '2935.png', '2967.png', '2948.png', '2943.png', '2953.png', '2963.png', '2968.png', '2945.png', '2981.png', '2993.png', '3011.png', '2986_2.png', '2988.png', '3003.png', '3016.png', '3001.png', '3021.png', '3026.png', '3020.png', '3008.png', '3024.png', '3029.png', '2994.png', '3025.png', '3006.png', '2995.png', '3026_2.png', '3033.png', '2998.png', '3000.png', '3012.png', '3014.png', '2996.png', '3030.png', '3017.png', '3019.png', '3031.png', '3032.png', '3002.png', '2992.png', '3010.png', '3004.png', '3013.png', '3015.png', '3027.png', '3005.png', '2985.png', '3009.png', '3022.png', '3018.png', '3028.png', '2997.png', '2991.png', '2986.png', '2999.png', '3023.png', '2989.png', '2983.png', '2984.png', '3045.png', '3043.png', '3046.png', '3061.png', '3034.png', '3067.png', '3049.png', '3073.png', '3037.png', '3081.png', '3075.png', '3072.png', '3078.png',

In [ ]:
import torchxrayvision as xrv
import skimage, torch, torchvision


# Prepare the image:
def create_findings(image_path):
  img = skimage.io.imread(image_path)
  # img = skimage.io.imread("image.png")
  # img = skimage.io.imread("CXR1_1_IM-0001-3001.png")
  img = xrv.datasets.normalize(img, 255) # convert 8-bit image to [-1024, 1024] range
  img = img.mean(2)[None, ...] # Make single color channel

  transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(224)])

  img = transform(img)
  img = torch.from_numpy(img)

  # Load model and process image
  model = xrv.models.DenseNet(weights="densenet121-res224-all")
  outputs = model(img[None,...]) # or model.features(img[None,...])

  # Print results
  # print(dict(zip(model.pathologies,outputs[0].detach().numpy())))
  data = dict(zip(model.pathologies,outputs[0].detach().numpy()))

  high_chance_conditions = [condition for condition, probability in data.items() if probability >= 0.75]
  moderate_chance_conditions = [condition for condition, probability in data.items() if 0.6 <= probability < 0.75]


  # Generate the summary
  summary_text = "The imaging analysis reveals "
  if high_chance_conditions or moderate_chance_conditions:
      if high_chance_conditions:
          summary_text += f"a high likelihood for the following conditions: {', '.join(high_chance_conditions)},\n"
      if moderate_chance_conditions:
          summary_text += f"and a moderate likelihood for the following conditions: {', '.join(moderate_chance_conditions)}.\n"
  else:
      summary_text = "No significant abnormalities detected from the imaging analysis. Findings appear within normal limits."

  # Output the summary
  # print("Generated Summary:")
  # print(summary_text)
  return summary_text

# {'Atelectasis': 0.32797316,
#  'Consolidation': 0.42933336,
#  'Infiltration': 0.5316924,
#  'Pneumothorax': 0.28849724,
#  'Edema': 0.024142697,
#  'Emphysema': 0.5011832,
#  'Fibrosis': 0.51887786,
#  'Effusion': 0.27805611,
#  'Pneumonia': 0.18569896,
#  'Pleural_Thickening': 0.24489835,
#  'Cardiomegaly': 0.3645515,
#  'Nodule': 0.68982,
#  'Mass': 0.6392845,
#  'Hernia': 0.00993878,
#  'Lung Lesion': 0.011150705,
#  'Fracture': 0.51916164,
#  'Lung Opacity': 0.59073937,
#  'Enlarged Cardiomediastinum': 0.27218717}


In [ ]:
# img = skimage.io.imread("/content/drive/My Drive/NLMCXR/images/CXR1_1_IM-0001-3001.png")

text = create_findings("/content/drive/My Drive/NLMCXR/frontal_images/4.png")
print(text)

# file_path = '/content/drive/My Drive/NLMCXR/gen_findings/1.txt'

# # Write data to a file
# with open(file_path, 'w') as f:
#     f.write(text)

{'Atelectasis': 0.5939554, 'Consolidation': 0.70307875, 'Infiltration': 0.6297401, 'Pneumothorax': 0.5768263, 'Edema': 0.51760423, 'Emphysema': 0.5137038, 'Fibrosis': 0.5527226, 'Effusion': 0.69844544, 'Pneumonia': 0.5677116, 'Pleural_Thickening': 0.5721153, 'Cardiomegaly': 0.5033054, 'Nodule': 0.54816806, 'Mass': 0.53445154, 'Hernia': 0.06515009, 'Lung Lesion': 0.53970194, 'Fracture': 0.5549015, 'Lung Opacity': 0.75482816, 'Enlarged Cardiomediastinum': 0.51546395}
The imaging analysis reveals a high likelihood for the following conditions: Lung Opacity,
and a moderate likelihood for the following conditions: Consolidation, Infiltration, Effusion.



In [ ]:
import os

# Define the folder paths
frontal_images_path = "/content/drive/My Drive/NLMCXR/frontal_images/"
lateral_images_path = "/content/drive/My Drive/NLMCXR/lateral_images/"
output_folder_path = '/content/drive/My Drive/NLMCXR/gen_findings_new/'
# folder_path_frontal = '/content/drive/My Drive/NLMCXR/frontal_images'
# folder_path_lateral = '/content/drive/My Drive/NLMCXR/lateral_images'


for i in range(1, 4000):
    # Construct file names
    frontal_file = os.path.join(frontal_images_path, f"{i}.png")
    lateral_file = os.path.join(lateral_images_path, f"{i}.png")
    # print(frontal_file)
    # Check existence and select the file
    if os.path.exists(frontal_file):
        chosen_file = frontal_file
    elif os.path.exists(lateral_file):
        chosen_file = lateral_file
    else:
        chosen_file = None

    # Print or process the chosen file
    if chosen_file:
      text = create_findings(chosen_file)

      # Define the output file path for saving the findings
      output_file_path = os.path.join(output_folder_path, f'{i}.txt')
      # print(output_file_path)

      # Write the findings text to the file
      with open(output_file_path, 'w') as f:
          f.write(text)

      print(f"Findings saved as {i}.txt")



Findings saved as 1.txt
Findings saved as 2.txt
Findings saved as 3.txt
Findings saved as 4.txt
Findings saved as 5.txt
Findings saved as 6.txt
Findings saved as 7.txt
Findings saved as 8.txt
Findings saved as 9.txt
Findings saved as 10.txt
Findings saved as 11.txt
Findings saved as 12.txt
Findings saved as 13.txt
Findings saved as 14.txt
Findings saved as 15.txt
Findings saved as 16.txt
Findings saved as 17.txt
Findings saved as 18.txt
Findings saved as 19.txt
Findings saved as 20.txt
Findings saved as 21.txt
Findings saved as 22.txt
Findings saved as 23.txt
Findings saved as 24.txt
Findings saved as 25.txt
Findings saved as 26.txt
Findings saved as 27.txt
Findings saved as 28.txt
Findings saved as 29.txt
Findings saved as 30.txt
Findings saved as 31.txt
Findings saved as 32.txt
Findings saved as 33.txt
Findings saved as 34.txt
Findings saved as 35.txt
Findings saved as 36.txt
Findings saved as 37.txt
Findings saved as 38.txt
Findings saved as 39.txt
Findings saved as 40.txt
Findings 

In [ ]:
import os

# Define the folder paths
images_folder_path = '/content/drive/My Drive/NLMCXR/images'
output_folder_path = '/content/drive/My Drive/NLMCXR/gen_findings/'

# Function to create findings (Assumed you already have this function defined)
# text = create_findings("/content/drive/My Drive/NLMCXR/images/CXR1_1_IM-0001-3001.png")

# List all files in the images folder
files = os.listdir(images_folder_path)

# Dictionary to store the first image for each prefix
first_images = {}

# Iterate through all files
for file_name in sorted(files):  # Sort to ensure the order
    # Split the file name to get the prefix (e.g., CXR1)
    prefix = file_name.split('_')[0]
    # print(file_name,prefix)
    # If the prefix is not already in the dictionary, add this file
    if prefix not in first_images:
        first_images[prefix] = file_name
        # print(prefix,file_name)

# Iterate through the first images for each prefix
for index, (prefix, file_name) in enumerate(first_images.items(), start=1):
    # Create the full file path for the image
    image_path = os.path.join(images_folder_path, file_name)
    # print(image_path)

    # Generate the findings text for the image
    text = create_findings(image_path)

    # Define the output file path for saving the findings
    number = ''.join(filter(str.isdigit, prefix))
    output_file_path = os.path.join(output_folder_path, f'{number}.txt')
    # print(output_file_path)

    # Write the findings text to the file
    with open(output_file_path, 'w') as f:
        f.write(text)

    print(f"Findings for {prefix} saved as {number}.txt")

print("All findings saved.")

Findings for CXR1000 saved as 1000.txt
Findings for CXR1001 saved as 1001.txt
Findings for CXR1002 saved as 1002.txt
Findings for CXR1003 saved as 1003.txt
Findings for CXR1004 saved as 1004.txt
Findings for CXR1005 saved as 1005.txt
Findings for CXR1006 saved as 1006.txt
Findings for CXR1007 saved as 1007.txt
Findings for CXR1008 saved as 1008.txt
Findings for CXR1009 saved as 1009.txt
Findings for CXR100 saved as 100.txt
Findings for CXR1010 saved as 1010.txt
Findings for CXR1011 saved as 1011.txt
Findings for CXR1012 saved as 1012.txt
Findings for CXR1013 saved as 1013.txt
Findings for CXR1014 saved as 1014.txt
Findings for CXR1015 saved as 1015.txt
Findings for CXR1016 saved as 1016.txt
Findings for CXR1017 saved as 1017.txt
Findings for CXR1018 saved as 1018.txt
Findings for CXR1019 saved as 1019.txt
Findings for CXR101 saved as 101.txt
Findings for CXR1020 saved as 1020.txt
Findings for CXR1021 saved as 1021.txt
Findings for CXR1022 saved as 1022.txt
Findings for CXR1023 saved as

OSError: Could not find a backend to open `/content/drive/My Drive/NLMCXR/images/Thumbs.db`` with iomode `r`.

In [ ]:
prefix = "CXR1"
number = ''.join(filter(str.isdigit, prefix))  # Extracts only the digits
print(number)

1
